In [2]:
import pandas as pd

In [3]:
from fuzzywuzzy import process

In [4]:
import random

In [5]:
import config

In [6]:
import pickle

In [7]:
from IPython.display import IFrame

In [8]:
import spotipy
import pandas as pd
import json
from spotipy.oauth2 import SpotifyClientCredentials


#Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= config.client_id,
                                                           client_secret= config.client_secret))

In [9]:
# Load the model from the pickle file
with open('kmeans_15.pkl', 'rb') as file:
    kmeans_15 = pickle.load(file)

In [10]:
with open('minmax.pkl', 'rb') as file:
    minmax = pickle.load(file)

In [11]:
url_1 = "/Users/karollvivianalopezvillegas/GitHub/DA_FT_Extra_Week10/Week_10/Mini-project_Spotify/Billboard_100.csv"

In [12]:
df_b = pd.read_csv(url_1)

In [14]:
bb_100 = df_b['song'].tolist()

In [15]:
clusters = "/Users/karollvivianalopezvillegas/GitHub/DA_FT_Extra_Week10/Week_10/Mini-project_Spotify/My_Spotify_selection_WITH CLUSTERS_15.csv" 

In [16]:
ds = pd.read_csv(clusters)

In [17]:
def song_recommender(score_corte=85):
    
    def play_song(track_id):
        return IFrame(src="https://open.spotify.com/embed/track/"+track_id,
           width="320",
           height="80",
           frameborder="0",
           allowtransparency="true",
           allow="encrypted-media",
          )

    name = input("Please enter the name of a song:").strip()
    coincidence = process.extractOne(name, bb_100, score_cutoff=score_corte)
    
    results_name = sp.search(q= name,limit=1)
    track_id_name = results_name["tracks"]["items"][0]["id"]
    
    display(play_song(track_id_name))
    
    
    if coincidence:
        # Encontró una coincidencia, obtener el nombre de la canción
        matched_song = coincidence[0]
        
        # Filtrar el DataFrame para encontrar la canción y el artista
        song_info = df_b[df_b['song'] == matched_song]

        # Recomendar una canción aleatoria del DataFrame
        recommended_song = df_b.sample(n=1).iloc[0]
        
        # Encontrar id de la canción recomendada:
        results = sp.search(q=recommended_song,limit=1)
        track_id = results["tracks"]["items"][0]["id"]
        
        print (f"Your meant this song name, right?: {matched_song}")
        print(f"Your song is popular! Here's another popular song: {recommended_song['song']}, Artist: {recommended_song['artist']}")
    
    else:
        # No encontró una coincidencia
        results = sp.search(q = name,limit=1)
        track_id = results["tracks"]["items"][0]["id"]
        name_features = sp.audio_features(track_id)
        df_name = pd.DataFrame(name_features)
        X_name = df_name[['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
        'duration_ms', 'time_signature']]
        X_name_scaled = minmax.fit_transform(X_name)
        cluster_sample = kmeans_15.predict(X_name_scaled)
        recommended_song = ds[ ds["cluster"] == cluster_sample.item() ].sample()
        track_id_recom = recommended_song["id"]
        track_id = track_id_recom.item()
        
    return play_song(track_id)

In [18]:
song_recommender()

Please enter the name of a song:hello


/opt/anaconda3/lib/python3.11/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(
